# Introduction to ITK Segmentation in SimpleITK Notebooks

<b>Goal</b>: 熟悉ITK中可用的基本分段算法，并以交互方式探索其参数空间。
图像分割过滤器处理图像以将其划分为（希望）有意义的区域。 输出通常是整数图像，其中每个整数可以表示一个对象。 值0通常用于背景，1（有时255）用于前景对象。


In [1]:
from __future__ import print_function

%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

import SimpleITK as sitk

# Download data to work on
%run update_path_to_download_script
from downloaddata import fetch_data as fdata
from myshow import myshow, myshow3d

### RescaleIntensity(Image image1, double outputMinimum=0, double outputMaximum=255) -> Image

对输入图像的强度级别应用线性变换。

参数：
- image1:需要进行强度变换的图像
- outputMinimum:变换后强度最小值,默认0.0
- outputMaximum:变换后强度最大值，默认255.0

返回:
强度变换后的图像

In [2]:
img_T1 = sitk.ReadImage(fdata("nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT1.nrrd"))
img_T2 = sitk.ReadImage(fdata("nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT2.nrrd"))
print("img_T1 size:{}".format(img_T1.GetSize()))

# To visualize the labels image in RGB with needs a image with 0-255 range
img_T1_255 = sitk.Cast(sitk.RescaleIntensity(img_T1), sitk.sitkUInt8)
img_T2_255 = sitk.Cast(sitk.RescaleIntensity(img_T2), sitk.sitkUInt8)

myshow3d(img_T1)

Fetching nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT1.nrrd
Fetching nac-hncma-atlas2013-Slicer4Version/Data/A1_grayT2.nrrd
img_T1 size:(288, 320, 208)
lenx:0,leny:0,lenz:0


interactive(children=(IntSlider(value=103, description='z', max=207), Output()), _dom_classes=('widget-interac…

## Thresholding

阈值处理是最基本的分割形式。 它只是根据强度范围标记图像的像素，而不考虑几何形状或连通3性。

### sitk.LabelOverlay(Image image, Image labelImage, double opacity=0.5, double backgroundValue=0.0, VectorUInt8 colormap) -> Image

将色彩表应用于label image并将其放在输入image之上。这组颜色是不同颜色的精选。 标签图像的不透明度可以由用户定义。 用户还可以选择是否要使用背景以及哪个标签值是背景。 背景标签产生的灰色像素的强度与输入的相同。
- backgroundValue-定义label image中作为背景像素的强度,默认是0

In [ ]:
help(sitk.LabelOverlay)

In [ ]:
#img_T1中体素值大于200的体素值1，否则为0，即生成一张label image
seg = img_T1>200
print(type(seg))

myshow(sitk.LabelOverlay(img_T1_255, seg,opacity=0.8,backgroundValue=0.0), "Basic Thresholding")

### BinaryThreshold(Image image1, double lowerThreshold=0.0, double upperThreshold=255.0, uint8_t insideValue=1, uint8_t outsideValue=0) -> Image

通过阈值处理将输入图像二值化,根据参数lowerThreshold和upperThreshold确定每个像素的二值化的值,详细说明如下:
$$
Output(x_{i})=
\left\{
\begin{matrix} InsideValue& LowerThreshold \leqslant x_{i} \leqslant UpperThreshold\hfill \\
OutsideValue & otherwise\hfill 
\end{matrix}
\right.
$$

In [ ]:
#像素值在100-400之间的像素设为1.0，之外的设为0.0
seg = sitk.BinaryThreshold(img_T1, lowerThreshold=100, upperThreshold=400, insideValue=1, outsideValue=0)

#下图中绿色区域表示像素在100-400之间
myshow(sitk.LabelOverlay(img_T1_255, seg), "Binary Thresholding")

ITK有许多基于直方图的自动阈值滤波器，包括``Huang``，``MaximumEntropy``，``Triangle``和流行的``Otsu``方法。 这些方法创建直方图，然后使用启发式来确定阈值。

### OtsuThreshold(Image image, Image maskImage, uint8_t insideValue=1, uint8_t outsideValue=0, uint32_t numberOfHistogramBins=128, bool maskOutput=True, uint8_t maskValue=255) -> Image

### OtsuThreshold(Image image, uint8_t insideValue=1, uint8_t outsideValue=0, uint32_t numberOfHistogramBins=128, bool maskOutput=True, uint8_t maskValue=255) -> Image

In [ ]:
otsu_filter = sitk.OtsuThresholdImageFilter()
otsu_filter.SetInsideValue(0)
otsu_filter.SetOutsideValue(1)
seg = otsu_filter.Execute(img_T1)
myshow(sitk.LabelOverlay(img_T1_255, seg), "Otsu Thresholding")

print(otsu_filter.GetThreshold())

## Region Growing Segmentation

在初始阈值处理上改进的第一步是一类称为区域增长的算法This includes:
<ul>
  <li><a href="http://www.itk.org/Doxygen/html/classitk_1_1ConnectedThresholdImageFilter.html">ConnectedThreshold</a></li>
  <li><a href="http://www.itk.org/Doxygen/html/classitk_1_1ConfidenceConnectedImageFilter.html">ConfidenceConnected</a></li>
  <li><a href="http://www.itk.org/Doxygen/html/classitk_1_1VectorConfidenceConnectedImageFilter.html">VectorConfidenceConnected</a></li>
  <li><a href="http://www.itk.org/Doxygen/html/classitk_1_1NeighborhoodConnectedImageFilter.html">NeighborhoodConnected</a></li>
</ul>

之前我们使用3D切片机来确定该索引：（132,142,96）是左侧脑室的良好种子。

#### BinaryDilate(Image arg1, uint32_t radius=1, itk::simple::KernelEnum kernel, double backgroundValue=0.0, double foregroundValue=1.0, bool boundaryToForeground=False) -> Image**
    
#### BinaryDilate(Image arg1, VectorUInt32 vectorRadius, itk::simple::KernelEnum kernel, double backgroundValue=0.0, double foregroundValue=1.0, bool boundaryToForeground=False) -> Image

In [ ]:
seed = (132,142,96)
seg = sitk.Image(img_T1.GetSize(), sitk.sitkUInt8)
seg.CopyInformation(img_T1)

#初始化1个点的像素值为1
seg[seed] = 1
seg = sitk.BinaryDilate(seg, 3)
myshow(sitk.LabelOverlay(img_T1_255, seg), "Initial Seed")

#### ConnectedThreshold(Image image1, VectorUIntList seedList, double lower=0, double upper=1, uint8_t replaceValue=1, itk::simple::ConnectedThresholdImageFilter::ConnectivityType connectivity) -> Image

标记连接到种子并位于值范围内的像素。 ConnectedThresholdImageFilter使用ReplaceValue的像素标记，那些位于下限和上限阈值范围内的连接到像素初始种子

In [ ]:
seg = sitk.ConnectedThreshold(img_T1, seedList=[seed], lower=100, upper=190)

myshow(sitk.LabelOverlay(img_T1_255, seg), "Connected Threshold")

对此进行改进的是``ConfidenceConnected``滤波器，它使用初始种子或当前分割来估计阈值范围。

#### ConfidenceConnected(Image image1, VectorUIntList seedList, unsigned int numberOfIterations=4, double multiplier=4.5, unsigned int initialNeighborhoodRadius=1, uint8_t replaceValue=1) -> Image

Segment pixels with similar statistics using connectivity.  This filter extracts a connected set of pixels whose pixel intensities are consistent with the pixel statistics of a seed point. The mean and variance across a neighborhood (8-connected, 26-connected, etc.) are calculated for a seed point. Then pixels connected to this seed point whose values are within the confidence interval for the seed point are grouped. The width of the confidence interval is controlled by the "Multiplier" variable (the confidence interval is the mean plus or minus the "Multiplier" times the standard deviation). If the intensity variations across a segment were gaussian, a "Multiplier" setting of 2.5 would define a confidence interval wide enough to capture 99% of samples in the segment.  After this initial segmentation is calculated, the mean and variance are re-calculated. All the pixels in the previous segmentation are used to calculate the mean the standard deviation (as opposed to using the pixels in the neighborhood of the seed point). The segmentation is then recalculated using these refined estimates for the mean and variance of the pixel values. This process is repeated for the specified number of iterations. Setting the "NumberOfIterations" to zero stops the algorithm after the initial segmentation from the seed point.  NOTE: the lower and upper threshold are restricted to lie within the valid numeric limits of the input data pixel type. Also, the limits may be adjusted to contain the seed point's intensity.

使用connectivity分割具有类似统计信息的像素。该滤波器提取连接的一组像素，他们的像素强度与种子点的像素统计一致。针对种子点计算邻域的平均值和方差（8连通，26连通等）。然后对连接到该种子点的像素进行分组，该种子点的值在种子点的置信区间内。置信区间的宽度由“Multiplier”变量控制（置信区间是平均值加上或减去“Multiplier”乘以标准差）。如果分割图上的强度变化是高斯的，则“Multiplier”设置为2.5将定义足够宽的置信区间以捕获段中99％的样本。在计算初始分割之后，重新计算均值和方差。先前分割中的所有像素用于计算标准偏差的平均值（与使用种子点附近的像素相反）。然后使用这些对像素值的均值和方差的精确估计来重新计算分割。对于指定的迭代次数重复此过程。将“NumberOfIterations”设置为零会在从种子点进行初始分割后停止算法。注意：下限阈值和上限阈值限制在输入数据像素类型的有效数字限制范围内。此外，可以调整限制以包含种子点的强度。

In [ ]:
seg = sitk.ConfidenceConnected(img_T1, seedList=[seed],
                                   numberOfIterations=1,
                                   multiplier=2.5,
                                   initialNeighborhoodRadius=1,
                                   replaceValue=1)

myshow(sitk.LabelOverlay(img_T1_255, seg), "ConfidenceConnected")

In [ ]:
img_multi = sitk.Compose(img_T1, img_T2)
seg = sitk.VectorConfidenceConnected(img_multi, seedList=[seed],
                                             numberOfIterations=1,
                                             multiplier=2.5,
                                             initialNeighborhoodRadius=1)
myshow(sitk.LabelOverlay(img_T2_255, seg))

## Fast Marching Segmentation

The FastMarchingImageFilter implements a fast marching solution to a simple level set evolution problem (eikonal equation). In this example, the speed term used in the differential equation is provided in the form of an image. The speed image is based on the gradient magnitude and mapped with the bounded reciprocal $1/(1+x)$.


In [ ]:
seed = (132,142,96)
feature_img = sitk.GradientMagnitudeRecursiveGaussian(img_T1, sigma=.5)
speed_img = sitk.BoundedReciprocal(feature_img) # This is parameter free unlike the Sigmoid
myshow(speed_img)

The output of the FastMarchingImageFilter is a <b>time-crossing map</b> that indicates, for each pixel, how much time it would take for the front to arrive at the pixel location.

In [ ]:
fm_filter = sitk.FastMarchingBaseImageFilter()
fm_filter.SetTrialPoints([seed])
fm_filter.SetStoppingValue(1000)
fm_img = fm_filter.Execute(speed_img)
myshow(sitk.Threshold(fm_img,
                    lower=0.0,
                    upper=fm_filter.GetStoppingValue(),
                    outsideValue=fm_filter.GetStoppingValue()+1))

In [ ]:
def fm_callback(img, time, z):
    seg = img<time;
    myshow(sitk.LabelOverlay(img_T1_255[:,:,z], seg[:,:,z]))
           
interact( lambda **kwargs: fm_callback(fm_img, **kwargs),
            time=FloatSlider(min=0.05, max=1000.0, step=0.05, value=100.0),
            z=(0,fm_img.GetSize()[2]-1))


## Level-Set Segmentation

There are a variety of level-set based segmentation filter available in ITK:
<ul>
<li><a href="http://www.itk.org/Doxygen/html/classitk_1_1GeodesicActiveContourLevelSetImageFilter.html">GeodesicActiveContour</a></li>
<li><a href="http://www.itk.org/Doxygen/html/classitk_1_1ShapeDetectionLevelSetImageFilter.html">ShapeDetection</a></li>
<li><a href="http://www.itk.org/Doxygen/html/classitk_1_1ThresholdSegmentationLevelSetImageFilter.html">ThresholdSegmentation</a></li>
<li><a href="http://www.itk.org/Doxygen/html/classitk_1_1LaplacianSegmentationLevelSetImageFilter.html">LaplacianSegmentation</a></li>
<li><a href="http://www.itk.org/Doxygen/html/classitk_1_1ScalarChanAndVeseDenseLevelSetImageFilter.html">ScalarChanAndVese</a></li>
</ul>

There is also a <a href="http://www.itk.org/Doxygen/html/group__ITKLevelSetsv4.html">modular Level-set framework</a> which allows composition of terms and easy extension in C++.




First we create a label image from our seed.

In [ ]:
seed = (132,142,96)

seg = sitk.Image(img_T1.GetSize(), sitk.sitkUInt8)
seg.CopyInformation(img_T1)
seg[seed] = 1
seg = sitk.BinaryDilate(seg, 3)

Use the seed to estimate a reasonable threshold range.

In [ ]:
stats = sitk.LabelStatisticsImageFilter()
stats.Execute(img_T1, seg)

factor = 3.5
lower_threshold = stats.GetMean(1)-factor*stats.GetSigma(1)
upper_threshold = stats.GetMean(1)+factor*stats.GetSigma(1)
print(lower_threshold,upper_threshold)

In [ ]:
init_ls = sitk.SignedMaurerDistanceMap(seg, insideIsPositive=True, useImageSpacing=True)

In [ ]:
lsFilter = sitk.ThresholdSegmentationLevelSetImageFilter()
lsFilter.SetLowerThreshold(lower_threshold)
lsFilter.SetUpperThreshold(upper_threshold)
lsFilter.SetMaximumRMSError(0.02)
lsFilter.SetNumberOfIterations(1000)
lsFilter.SetCurvatureScaling(.5)
lsFilter.SetPropagationScaling(1)
lsFilter.ReverseExpansionDirectionOn()
ls = lsFilter.Execute(init_ls, sitk.Cast(img_T1, sitk.sitkFloat32))
print(lsFilter)

In [ ]:
myshow(sitk.LabelOverlay(img_T1_255, ls>0))